Generate Fake data for testing using below code

In [ ]:
import numpy as np
import pandas as pd
TARGET_LABEL = 'transient_type'

def generate_data(original, num_datapoints):
    data = []
    for i in range(num_datapoints):
        noise = np.random.normal(loc=0, scale=0.1, size=3)
        data.append(original + noise)
        
        
    return np.array(data)
num_type1 = 40
num_type2 = 20
c1 = generate_data([0.2, 0.1, .9], num_type1)
c2 = generate_data([0.1, 0.2, .8], num_type2)
data_X = np.concatenate((c1,c2))
fake_X = pd.DataFrame(data_X, columns = ['f1', 'f2', 'f3'])
fake_Y = pd.DataFrame(["Ia"]*num_type1 + ["CC"]*num_type2 , columns = [TARGET_LABEL])

fake_data = [fake_X, fake_Y]

# fake_X.f1[0]  = None
# fake_X.f1[1]  = None
# for i in range(num_type1):
#     fake_X.f1[i]  = None

## Run model

In [ ]:
Ia_Rate = 200000
II_Rate = 100000
TDE_Rate = 6000
GRB_Rate = 50

import numpy as np
rates = np.array([Ia_Rate, II_Rate, TDE_Rate, GRB_Rate])
rates = rates/sum(rates)
class_rates = rates.tolist()
class_rates

In [ ]:
import warnings
warnings.filterwarnings('ignore')
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
%matplotlib inline 
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)


from models.binary_model.binary_model import BinaryModel
from models.ind_model.ind_model import IndModel
from models.multi_model.multi_model import MultiModel

import matplotlib.pyplot as plt

# Turn interactive plotting off
plt.ioff()


# mags = ["g_mag",  "r_mag", "i_mag", "z_mag", "y_mag", "u_mag",
#         "W1_mag", "W2_mag", "W3_mag", "W4_mag",
#         "J_mag", "K_mag", "H_mag",
#         "NUV_mag", "FUV_mag"]

mags = ["g_mag",  "r_mag", "i_mag", "z_mag", "y_mag",
        "W1_mag", "W2_mag",
        "J_mag", "K_mag", "H_mag"]

runs = 2
model = MultiModel(
#     nb= True,
       folds = 5,
       min_class_size = 25,
#        max_class_size = 100,
       class_labels = ["Unspecified Ia", "Unspecified II",  "GRB"],
#        priors = class_rates,
       transform_features = False,
       cols = mags,
        )


model.analyze_probabilities()
# model.run_model()

## Load previous run results

In [ ]:
# Rerun performance visualizations on saved output of model. 

import pickle
pickle_dir = "/Users/marina/Documents/PhD/research/astro_research/code/thex_model/output/Multiclass_Classifier7/"

with open(pickle_dir + 'results.pickle', 'rb') as handle:
    results = pickle.load(handle)
    
with open(pickle_dir + 'y.pickle', 'rb') as handle:
    y = pickle.load(handle)   
    
    
model.results = results
model.y = y


# model.visualize_performance()

## Test new density code

In [ ]:
results = np.concatenate(model.results)

model.plot_density_half_compare(results)

## Get LSST daily estimates

In [ ]:
approved_ranges = {'Ia' : [14.39, 15.22],
         'II' : [15.52, 17.13],
         'TDE' : [16.63, 21.17]}

In [ ]:
keep_indices = []
for index, row in model.X.iterrows():
    y_row = model.y.iloc[index]
    label = y_row['transient_type']
    for class_name in approved_ranges.keys():
        if class_name in label:
            min_range = approved_ranges[class_name][0]
            max_range = approved_ranges[class_name][1]
            if row['r_mag'] >= min_range and row['r_mag'] <= max_range:
                keep_indices.append(index)

## Examine example outputs, with and without priors

In [ ]:
class_indices_map = {}

ranges = {'Ia' : [14.39, 15.22],
         'II' : [15.52, 17.13],
         'TDE' : [16.63, 21.17]}

In [ ]:
priors = True

cur_class = 'GRB'

indices = []
for index, row in model.X.iterrows():
    if cur_class in ranges.keys():
        min_val = ranges[cur_class][0]
        max_val = ranges[cur_class][1]
        r = row['r_mag']
        y_example=model.y.iloc[index]
        if r >= min_val and r <= max_val and cur_class in y_example['transient_type']:
            indices.append(index)
    else:
        y_example=model.y.iloc[index]
        if  cur_class in y_example['transient_type']:
            indices.append(index)
indices


In [ ]:
# model.model.clf.class_priors = class_rates
# model.model.clf.class_priors = None
# model.model.clf.class_priors 

In [ ]:
import random
num_samples = 10
class_indices = []

for i in range(num_samples):
    r = random.randint(0, len(indices))
    index = indices[r]
    class_indices.append(index)
    
    X_example = model.X.iloc[index]
    p = model.get_class_probabilities(X_example)
    ps = list(p.values())
    y_example = model.y.iloc[index]

    r = np.hstack((ps, y_example.values))
    model.plot_example_output(r, index, priors)


class_indices_map[cur_class] = class_indices

In [ ]:
# Run on same indices with/without priors (opposite of whatever was done last time)

priors=False
cur_class = "GRB"
class_indices = class_indices_map[cur_class] 

for index in class_indices:    
    X_example = model.X.iloc[index]
    p = model.get_class_probabilities(X_example)
    ps = list(p.values())
    y_example = model.y.iloc[index]

    r = np.hstack((ps, y_example.values))
    model.plot_example_output(r, index, priors)



In [ ]:
PATH = "/Users/marina/Documents/PhD/research/astro_research/code/dist_code/data/"
# data = "all_features"
data = "g_W2"
model.X.to_csv(PATH+ data + "_X.csv")
model.y.to_csv(PATH+ data + "_y.csv")